.64

# Entrenando modelo BETO de detecta premisas y conclusiones en oraciones con PoC

# Introduction

# Setup

**Montar disco de Gdrive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE

fold_0_test.txt   fold_2_train.txt  fold_5_test.txt   fold_7_train.txt	 fold_9_train.txt
fold_0_train.txt  fold_3_test.txt   fold_5_train.txt  fold_8_test.txt
fold_1_test.txt   fold_3_train.txt  fold_6_test.txt   fold_8_train.txt
fold_1_train.txt  fold_4_test.txt   fold_6_train.txt  fold_9_test.txt
fold_2_test.txt   fold_4_train.txt  fold_7_test.txt   fold_9_train.gdoc


Download `transformers` and install required packages.

In [8]:
#%%capture
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..


Cloning into 'transformers'...
remote: Enumerating objects: 224049, done.
remote: Counting objects: 100% (687/687), done.
remote: Compressing objects: 100% (452/452), done.
remote: Total 224049 (delta 372), reused 385 (delta 184), pack-reused 223362 (from 1)
Receiving objects: 100% (224049/224049), 239.10 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (162489/162489), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9531229 sha256=4203a302b7d24ea122350fa76b4a656b5d0479c3ee8e08d96eb2f240a2846135
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ndmzuua/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalli

In [10]:
#usar info de huggigfaces
# https://github.com/huggingface/transformers/tree/master/examples/token-classification

# !wget -O 'requirements_tokenclasify.txt' "https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/requirements.txt"

# !pip install -r "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/requirements.txt"
!pip install -r "/content/drive/MyDrive/ColabNotebooks/BERT/Scripts/requirements.txt"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

# Data


## 1. Download Datasets

Para deteccion de componentes

   - dev.txt: Spanish test data for the development stage
   - test.txt: Spanish test data
   - train.txt: Spanish train data

In [11]:
!ls /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE

fold_0_test.txt   fold_2_train.txt  fold_5_test.txt   fold_7_train.txt	 fold_9_train.txt
fold_0_train.txt  fold_3_test.txt   fold_5_train.txt  fold_8_test.txt
fold_1_test.txt   fold_3_train.txt  fold_6_test.txt   fold_8_train.txt
fold_1_train.txt  fold_4_test.txt   fold_6_train.txt  fold_9_test.txt
fold_2_test.txt   fold_4_train.txt  fold_7_test.txt   fold_9_train.gdoc


## 2. Preprocessing - quizas usar otro modelo bert-base-multilingual-cased

Let's define some variables that we need for further pre-processing steps and training the model:

In [12]:
MAX_LENGTH = 120 #@param {type: "integer"}
MODEL = "dccuchile/bert-base-spanish-wwm-uncased"  #@param ["chriskhanhtran/spanberta", "bert-base-multilingual-cased","dccuchile/bert-base-spanish-wwm-uncased", "pysentimiento/robertuito-base-uncased"]

# dccuchile/bert-base-spanish-wwm-uncased

The script below will split sentences longer than `MAX_LENGTH` (in terms of tokens) into small ones. Otherwise, long sentences will be truncated when tokenized, causing the loss of training data and some tokens in the test set not being predicted.

In [13]:
%%capture
# !wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

# !cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/preprocess.py" "/content/"
!cp "/content/drive/MyDrive/ColabNotebooks/BERT/Scripts/preprocess.py" "/content/"

In [14]:
!ls

drive  preprocess.py  sample_data  transformers


In [15]:
# !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_compOracionesArgC_folds/fold_0_train.txt $MODEL $MAX_LENGTH > train.txt
# # !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_comp/fold_0_dev.txt $MODEL $MAX_LENGTH > dev.txt
# !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_compOracionesArgC_folds/fold_0_test.txt $MODEL $MAX_LENGTH > test.txt


!python3 preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE/fold_0_train.txt $MODEL $MAX_LENGTH > train.txt
# # !python3 preprocess.py ./drive/MyDrive/experimentosArgComp/datos_comp/fold_0_dev.txt $MODEL $MAX_LENGTH > dev.txt
!python3 preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE/fold_0_test.txt $MODEL $MAX_LENGTH > test.txt

tokenizer_config.json: 100% 310/310 [00:00<00:00, 1.75MB/s]
config.json: 100% 650/650 [00:00<00:00, 4.92MB/s]
vocab.txt: 100% 248k/248k [00:00<00:00, 713kB/s]
tokenizer.json: 100% 486k/486k [00:00<00:00, 933kB/s]
special_tokens_map.json: 100% 134/134 [00:00<00:00, 767kB/s]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com

## 3. Labels

In comp, there are have X classes of NER tags:

- O, Outside of a component
- B-Arg, Beginning of arg
- I-Arg, arg entity


If your dataset has different labels or more labels than CoNLL-2002/2003 datasets, run the line below to get unique labels from your data and save them into `labels.txt`. This file will be used when we start fine-tuning our model.

In [16]:
!cat train.txt  test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt
# !cat train.txt dev.txt test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [17]:
!head -n20 labels.txt

B-QUE
I-QUE
O


In [18]:
!ls

drive  labels.txt  preprocess.py  sample_data  test.txt  train.txt  transformers


# Fine-tuning Model


These are the example scripts from `transformers`'s repo that we will use to fine-tune our model for NER. After 04/21/2020, Hugging Face has updated their example scripts to use a new `Trainer` class. To avoid any future conflict, let's use the version before they made these updates.

In [19]:
!pip install conllu

In [20]:
%%capture
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/run_ner.py"
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/utils_ner.py"
# !wget "https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/token-classification/tasks.py"


# !cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/run_ner.py" "/content/"
# !cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/tasks.py" "/content/"
# !cp "/content/drive/MyDrive/Exp_Objetivos2021/config_ner/utils_ner.py" "/content/"


!cp "/content/drive/MyDrive/ColabNotebooks/BERT/Scripts/run_ner.py" "/content/"
!cp "/content/drive/MyDrive/ColabNotebooks/BERT/Scripts/tasks.py" "/content/"
!cp "/content/drive/MyDrive/ColabNotebooks/BERT/Scripts/utils_ner.py" "/content/"



In [21]:
!ls

drive	    preprocess.py  sample_data	test.txt   transformers
labels.txt  run_ner.py	   tasks.py	train.txt  utils_ner.py


In [22]:
# !ls /content/drive/MyDrive/Exp_Objetivos2021/folders
!ls /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE

fold_0_test.txt   fold_2_train.txt  fold_5_test.txt   fold_7_train.txt	 fold_9_train.txt
fold_0_train.txt  fold_3_test.txt   fold_5_train.txt  fold_8_test.txt
fold_1_test.txt   fold_3_train.txt  fold_6_test.txt   fold_8_train.txt
fold_1_train.txt  fold_4_test.txt   fold_6_train.txt  fold_9_test.txt
fold_2_test.txt   fold_4_train.txt  fold_7_test.txt   fold_9_train.gdoc


In [23]:
# grid search
 # Model training
max_accuracy = 0

# epoch 4
epoch = [160]   # 20 , 80,  160,  200     - 2, 3, 5, 10, 30, 60, 100, 200
# batch sizes: 8, 16, 32, 64, 128
# learning rates: 3e-4, 1e-4, 5e-5, 3e-5

# learning_rate choices
# learning_rates = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.01, 0.02, 0.03, 0.04, 0.05 ]
learning_rates = [3e-5] #  3e-6

# iterations choices
# iterations = [ 100, 200, 300, 400, 500 ]
batch = [16] # 16,32

# #folds
# folds = list(range(10))

# available combination of learning_rate and iterations

parameters = []
for z  in epoch:
    for j in batch:
      for i in learning_rates:
        parameters.append( ( i, j, z) )

print("Available combinations : ",len(parameters),  parameters )


# Applying linear searching in list of available combination
# to achieved maximum accuracy on CV set


Available combinations :  1 [(3e-05, 16, 160)]


In [24]:
  # dir = "/content/drive/MyDrive/Exp_Objetivos2021/results/"
dir = "/content/drive/MyDrive/ColabNotebooks/BERT/Results/"

# /content/drive/MyDrive/experimentosArgComp/results/22abrCorpus5arg
# /content/drive/MyDrive/experimentosArgComp/sample_data
# /content/drive/MyDrive/experimentosArgComp/results/BETOcorpus4compPCsolo

f = open(dir+"log_exp.txt", "a+")
log_text = f"results\n"
f.write(log_text)
f.close()

In [25]:
!pip install TensorRT

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for TensorRT: filename=tensorrt-10.3.0-py2.py3-none-any.whl size=16337 sha256=2dc67d0e08eea04d6844ae9916e3bb884140253f86878b3a3c5af0b1883a7640
  Stored in directory: /root/.cache/pip/wheels/e8/a8/32/bacdf7c14f627d34b90ce8586631e2f196e3fb0fd4eb9add55
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.3.0-py2.py3-none-any.whl size=17554 sha256=72107b6379f48bb2a9b3ef164166bef837afc29311ebf3fd7430a1198173ad47
  Stored in directory: /root/.cache/pip/wheels/14/31/c2/a3ecb74def7087b76ef2a701e6823f03e40ab43348c668a571
Successfully built TensorRT tensorrt-cu12


In [26]:
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [27]:
def read_examples_from_file(file_path):
    """Read words and labels from a CoNLL-2002/2003 data file.

    Args:
      file_path (str): path to NER data file.

    Returns:
      examples (dict): a dictionary with two keys: `words` (list of lists)
        holding words in each sequence, and `labels` (list of lists) holding
        corresponding labels.
    """
    with open(file_path, encoding="utf-8") as f:
        examples = {"words": [], "labels": []}
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples["words"].append(words)
                    examples["labels"].append(labels)
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
    return examples

In [28]:
import time
from transformers import AutoTokenizer, AutoModelForTokenClassification
#model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# for i in range(100):
#   x = 100 + 11999 + 2000
#   current_time = time.time()
#   elapsed_time = current_time - start_time



# (0.0003, 8) learn y batch
#epoch = 10
corpus = "ArgPC"
max_test_f1 = 0
max_test_accuracy_score = 0
max_parametros = ""


for k in range( len( parameters ) ) :

  sumaF1 = 0
  sumaAcc = 0


  for fold in range(0,10):

    start_time = time.time()

    !rm train.txt
    !rm test.txt


    # !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_train.txt $MODEL $MAX_LENGTH > train.txt
    # !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt

    !python3 preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE/fold_{fold}_train.txt $MODEL $MAX_LENGTH > train.txt
    !python3 preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt


    !cat train.txt  test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

    #epoch = parameters[k][2]
    epoch=40



    MAX_LENGTH = 128
    MODEL = "dccuchile/bert-base-spanish-wwm-uncased"
    #MODEL = "chriskhanhtran/spanberta"
    OUTPUT_DIR = "bert-base-ml-ner"
    BATCH_SIZE = parameters[k][1] #32
    NUM_EPOCHS = epoch
    SAVE_STEPS = 100
    LOGGING_STEPS = 100
    SEED = 42
    LEARNING_RATE = parameters[k][0]


    parametros = f"epoch_{str(epoch)}_learn_{str(LEARNING_RATE)}_batch_{str(BATCH_SIZE)}_corpus{corpus}_fold{str(fold)}\n"

    !python3 run_ner.py --data_dir ./ \
    --labels ./labels.txt \
    --model_name_or_path $MODEL \
    --output_dir $OUTPUT_DIR \
    --max_seq_length  $MAX_LENGTH \
    --num_train_epochs $NUM_EPOCHS \
    --per_gpu_train_batch_size $BATCH_SIZE \
    --save_steps $SAVE_STEPS \
    --seed $SEED \
    --do_train \
    --do_predict \
    --overwrite_cache \
    --overwrite_output_dir \
    --save_total_limit 2 \
    --learning_rate $LEARNING_RATE




    !cp ./bert-base-ml-ner/config.json  {dir}config_{parametros}.json
    !cp ./bert-base-ml-ner/test_predictions.txt   {dir}test_predictions_{parametros}.txt
    !cp ./bert-base-ml-ner/test_results.txt   {dir}test_results_{parametros}.txt

    archivo_result = "./bert-base-ml-ner/test_results.txt"
    test_f1_current = 0
    test_accuracy_score_current = 0

    current_time = time.time()
    elapsed_time = current_time - start_time

    testsite_array = []
    with open(archivo_result) as my_file:
        for line in my_file:
            testsite_array.append(line.split() )
            if line.split()[0] == "test_f1":
              print("test_f1",line.split()[2],parametros)
              test_f1_current = float(line.split()[2])
            if line.split()[0] ==  "test_accuracy_score":
              print("test_accuracy_score",line.split()[2],parametros)
              test_accuracy_score_current = float(line.split()[2])

    f = open(dir+"log_exp.txt", "a+")
    log_text = f"{str(test_accuracy_score_current),str(test_f1_current)},{str(epoch)},{str(LEARNING_RATE)},{str(BATCH_SIZE)},{corpus},{str(fold)},time,{str(elapsed_time)}\n"
    f.write(log_text)
    f.close()

    # guardo para acumular F1 y acc

    sumaAcc += test_accuracy_score_current
    sumaF1 += test_f1_current


    if max_test_f1 < test_f1_current :
      max_test_f1 = test_f1_current
      max_parametros = parametros

    #**********resultados
    import numpy as np
    from sklearn.metrics import classification_report
    !rm test.txt

    # !python3 preprocess.py /content/drive/MyDrive/Exp_Objetivos2021/folders/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt
    !python3 preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/FoldersQUE/fold_{fold}_test.txt $MODEL $MAX_LENGTH > test.txt
    !python3 preprocess.py /content/bert-base-ml-ner/test_predictions.txt $MODEL $MAX_LENGTH > prediction.txt

    !cat test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt


    y_true = read_examples_from_file("test.txt")["labels"]
    y_pred = read_examples_from_file("prediction.txt")["labels"]
    print ("RESULTADOS FOLDER", fold)
    print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

    #***********resultados


  print(max_test_f1,max_parametros)








/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-08-15 14:18:03.739104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-

# Download Results

In [29]:
!zip -r /content/bert-base-ml-ner.zip /content/bert-base-ml-ner

  adding: content/bert-base-ml-ner/ (stored 0%)
  adding: content/bert-base-ml-ner/test_predictions.txt (deflated 70%)
  adding: content/bert-base-ml-ner/config.json (deflated 49%)
  adding: content/bert-base-ml-ner/tokenizer_config.json (deflated 75%)
  adding: content/bert-base-ml-ner/test_results.txt (deflated 43%)
  adding: content/bert-base-ml-ner/model.safetensors (deflated 7%)
  adding: content/bert-base-ml-ner/vocab.txt (deflated 56%)
  adding: content/bert-base-ml-ner/checkpoint-840/ (stored 0%)
  adding: content/bert-base-ml-ner/checkpoint-840/config.json (deflated 49%)
  adding: content/bert-base-ml-ner/checkpoint-840/model.safetensors (deflated 7%)
  adding: content/bert-base-ml-ner/checkpoint-840/optimizer.pt (deflated 27%)
  adding: content/bert-base-ml-ner/checkpoint-840/trainer_state.json (deflated 54%)
  adding: content/bert-base-ml-ner/checkpoint-840/scheduler.pt (deflated 56%)
  adding: content/bert-base-ml-ner/checkpoint-840/training_args.bin (deflated 51%)
  adding

In [ ]:
from google.colab import files
files.download("/content/bert-base-ml-ner.zip.zip")

# Tests


In [32]:
!echo "MODEL"         $MODEL
!echo "OUTPUT_DIR"    $OUTPUT_DIR
!echo "MAX_LENGTH"    $MAX_LENGTH
!echo "NUM_EPOCHS"    $NUM_EPOCHS
!echo "BATCH_SIZE"    $BATCH_SIZE
!echo "SAVE_STEPS"    $SAVE_STEPS
!echo "SEED"          $SEED
!echo "LEARNING_RATE" $LEARNING_RATE

MODEL dccuchile/bert-base-spanish-wwm-uncased
OUTPUT_DIR bert-base-ml-ner
MAX_LENGTH 128
NUM_EPOCHS 40
BATCH_SIZE 16
SAVE_STEPS 100
SEED 42
LEARNING_RATE 3e-05


In [34]:
!python preprocess.py /content/drive/MyDrive/ColabNotebooks/BERT/Tests/obetivo.txt /content/bert-base-ml-ner 128 > /content/drive/MyDrive/ColabNotebooks/BERT/Tests/test.txt

In [35]:
!cat /content/drive/MyDrive/ColabNotebooks/BERT/Tests/test.txt

implementar O
un O
esquema O
de O
base O
de O
datos O
que O
soporte O
seguimiento O
semanal O
y O
listas O
de O
tareas O
kanban O
. O


In [37]:
!python3 run_ner.py --data_dir /content/drive/MyDrive/ColabNotebooks/BERT/Tests \
    --labels /content/drive/MyDrive/ColabNotebooks/BERT/Tests/labels.txt \
    --model_name_or_path /content/bert-base-ml-ner \
    --output_dir /content/drive/MyDrive/ColabNotebooks/BERT/Tests/results \
    --max_seq_length  128 \
    --num_train_epochs 40 \
    --per_gpu_train_batch_size 16 \
    --save_steps 100 \
    --seed 42 \
    --do_predict \
    --overwrite_cache \
    --overwrite_output_dir \
    --save_total_limit 2 \
    --learning_rate 3e-05

2024-08-15 16:06:43.554409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 16:06:43.574256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 16:06:43.580332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 16:06:43.594939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 16:06:44.766577: W tensorflow/compiler/tf2